# FFF Workshop

## A5: Algorithmic fragment merging

### Outline

- Identifying merging opportunities
- Small scale Fragmenstein merge via API
- Small scale Knitwork
- Point to larger scale stuff in day 2
- Squonk jobs

## Identifying merging opportunities

For fragment progression it is often desirable to restrict the observations considered for merging. When working with Fragalysis data, tags and Canonical Sites assigned by XCA can be useful starting points

In [1]:
# setup the animal
%load_ext autoreload
%autoreload 2
import hippo
animal = hippo.HIPPO(
    "A71EV2A_demo",
    "../data/A71EV2A.sqlite",
)

 Creating HIPPO animal

name = A71EV2A_demo

db_path = ../data/A71EV2A.sqlite

DEBUG: hippo.Database.__init__()

DEBUG: Database.path = ../data/A71EV2A.sqlite

DEBUG: hippo.Database.connect()

DEBUG: sqlite3.version='2.6.0'

 Success  Database connected @ ../data/A71EV2A.sqlite!

 Success  Initialised animal HIPPO("A71EV2A_demo")!

For the example target A71EV2A it makes most sense to explore merges of fragments in the active site:

In [6]:
active_site_fragments = animal.poses(tag="[Other] Active site fragment")
active_site_fragments

poses tagged "[Other] Active site fragment": {P × 44}

In [5]:
active_site_hits = animal.poses(tag="hits").get_by_subsite(id=1)
active_site_hits

poses tagged "hits" & subsite=1: {P × 611}

You may also want to explore merges of other/larger ligands in the active site:

One way to anticipate pairs of molecules that are suitable for merging is to look at their interactions. An ideal merge will combine molecules that share some but not all interactions with each other. Consider the following pair:

In [9]:
animal.poses["A0207a", "A0237a"].draw()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The overlapping ring might make it suitable for merging. Looking at the interactions:

In [10]:
animal.poses["A0207a"].interactions.summary()
animal.poses["A0237a"].interactions.summary()

 {I × 3}

π-stacking [B PRO 107] 3.7 Å, 9.9 degrees = 3.7 Å

Hydrophobic [B ASP 109] 4.5 Å = 4.5 Å

Hydrophobic [B VAL 124] 3.5 Å = 3.5 Å

 {I × 6}

...

Hydrophobic [B GLU 106] 4.4 Å = 4.4 Å

Hydrophobic [B GLU 106] 4.1 Å = 4.1 Å

Hydrophobic [B PRO 107] 3.8 Å = 3.8 Å

π-stacking [B PRO 107] 4.1 Å, 6.3 degrees = 4.1 Å

Hydrophobic [B VAL 124] 3.9 Å = 3.9 Å

There are 2 shared interactions, which could be exploited during merging.

In HIPPO a `PoseSet` method exists to count these opportunities using interactions in the dataset:

In [31]:
animal.poses["A0207a", "A0237a"].get_interaction_overlaps()

1

This tells us that `A0207a` and `A0237a` might be worth trying to merge.

For a larger set you can count all the interaction overlaps as well:

In [33]:
active_site_fragments.get_interaction_overlaps()

288

You can also get the pairs as a list of `PoseSet` objects:

In [37]:
pairs = active_site_fragments.get_interaction_overlaps(return_pairs=True)

for pair in pairs:
    print(pair.names)
    break

['A0739a', 'A0351a']


## Fragmenstein Merging

Fragmenstein can be used to generate merges of multiple molecules, and minimise their conformations relative to the reference/inspiration molecules.

A Fragmenstein merging job is available in Fragalysis/Squonk and can be requested with the fragalysis Python API:

In [54]:
from fragalysis.requests.jobs import fragmenstein_combine

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 from fragalysis.requests.jobs import fragmenstein_combine                                    │
│   2                                                                                              │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │ active_site_fragments = poses tagged "[Other] Active site fragment": {P × 44}                │ │
│ │      active_site_hits = poses tagged "hits" & subsite=1: {P × 611}                           │ │
│ │                animal = HIPPO("A71EV2A_demo")                                                │ │
│ │                  exit = <IPython.core.autocall.ZMQExitAutocall object at 0x73984dd37b30>     │ │
│ │           get_ipython = <bound method InteractiveShell.get_ipython of                        │ │
│ │                         <ipykernel.zmqshell.ZMQInteractiveShell object at 0x73984c801160>>   │ │
│ │                 hippo = <module 'hippo' from '/home/jovyan/HIPPO/hippo/__init__.py'>         │ │
│ │                    In = [                                                                    │ │
│ │                         │   '',                                                              │ │
│ │                         │   "# setup the animal\nget_ipython().run_line_magic('load_ext',    │ │
│ │                         'autoreload')\nget_ip"+128,                                          │ │
│ │                         │   'active_site_hits = animal.poses("hits").get_by_subsite(1)',     │ │
│ │                         │   'active_site_hits = animal.poses(tag="hits").get_by_subsite(1)', │ │
│ │                         │   'active_site_hits =                                              │ │
│ │                         animal.poses(tag="hits").get_by_subsite(id=1)',                      │ │
│ │                         │   'active_site_hits =                                              │ │
│ │                         animal.poses(tag="hits").get_by_subsite(id=1)\nactive_site_hit'+1,   │ │
│ │                         │   'active_site_fragments = animal.poses(tag="[Other] Active site   │ │
│ │                         fragment")\nactive_'+14,                                             │ │
│ │                         │   'animal.poses["A0237a", "A0207a"].draw()',                       │ │
│ │                         │   'animal.poses["A0237a"].interactions.summary()',                 │ │
│ │                         │   'animal.poses["A0207a", "A0237a"].draw()',                       │ │
│ │                         │   ... +45                                                          │ │
│ │                         ]                                                                    │ │
│ │                 mrich = <module 'mrich' from                                                 │ │
│ │                         '/opt/conda/lib/python3.12/site-packages/mrich/__init__.py'>         │ │
│ │                   Out = {                                                                    │ │
│ │                         │   5: poses tagged "hits" & subsite=1: {P × 611},                   │ │
│ │                         │   6: poses tagged "[Other] Active site fragment": {P × 44},        │ │
│ │                         │   11: 2,                                                           │ │
│ │                         │   12: 576,                                                         │ │
│ │                         │   13: 2,                                                           │ │
│ │                         │   14: 2,                       